<a href="https://colab.research.google.com/github/MASAYA-KINOSHITA/first-Repository/blob/main/Nishika_%E3%82%BD%E3%83%95%E3%83%88%E3%82%A6%E3%82%A7%E3%82%A2%E7%95%B0%E5%B8%B8_xgb_testreloded6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ライブラリのインポート
import pandas as pd # 基本ライブラリ
import numpy as np # 基本ライブラリ
import matplotlib.pyplot as plt # グラフ描画用
import seaborn as sns; sns.set() # グラフ描画用
import warnings # 実行に関係ない警告を無視
warnings.filterwarnings('ignore')

!pip install optuna #optunaのインストール

import optuna.integration.lightgbm as lgb_tune #LightGBMのパラメータチューニング
import lightgbm as lgb #LightGBM
from sklearn import datasets
from sklearn.model_selection import train_test_split # データセット分割用
from sklearn.metrics import accuracy_score # モデル評価用(正答率)
from sklearn.metrics import log_loss # モデル評価用(logloss)     
from sklearn.metrics import roc_auc_score # モデル評価用(auc)
# 評価指標
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error



# データフレームを綺麗に出力する関数
import IPython
def display(*dfs, head=True):
    for df in dfs:
        IPython.display.display(df.head() if head else df)

     |████████████████████████████████| 308 kB 8.5 MB/s 
     |████████████████████████████████| 210 kB 55.3 MB/s 
     |████████████████████████████████| 81 kB 9.3 MB/s 
     |████████████████████████████████| 78 kB 7.2 MB/s 
     |████████████████████████████████| 49 kB 6.1 MB/s 
     |████████████████████████████████| 113 kB 62.4 MB/s 
     |████████████████████████████████| 150 kB 49.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=3381c48f339597e6b5f46da2f6092153b3c5099f8c16b55bc420852bcab4e78e
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [3]:
# data_train csvファイルの読み込み
train_df = pd.read_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/train.zip (Unzipped Files)/train.csv',usecols=['Anomaly',
 'GC activity : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionCount))',
 'GC time : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionTime))',
 'Rel. swap usage : ((MXBean(java.lang:type=OperatingSystem).FreeSwapSpaceSize / MXBean(java.lang:type=OperatingSystem).TotalSwapSpaceSize))',
 'Stuck threads : (MXBean(com.bea:Name=ThreadPoolRuntime,Type=ThreadPoolRuntime).StuckThreadCount)',
 'Stuck threads : (MXBean(com.bea:ApplicationRuntime=sys01,Name=default,Type=WorkManagerRuntime).StuckThreadCount)',
 'Process CPU : (MXBean(java.lang:type=OperatingSystem).ProcessCpuLoad)',
 'Rel. heap committed : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.committed / MXBean(java.lang:type=Memory).HeapMemoryUsage.max))',
 'System CPU : (MXBean(java.lang:type=OperatingSystem).SystemCpuLoad)',
 'Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))',
 'Process CPU : (\\Process(java)\\CPU)',
 'Rel. heap usage : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.used / MXBean(java.lang:type=Memory).HeapMemoryUsage.max))',
 'Rel. open file descriptors : ((MXBean(java.lang:type=OperatingSystem).OpenFileDescriptorCount / MXBean(java.lang:type=OperatingSystem).MaxFileDescriptorCount))',
 'Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))',
 'Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))',
 'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
 'Memory space usage : ((MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.committed / MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.max))',
 'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
 'Memory space usage : ((MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.max))',
 'Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))',
 'Daemon thread count : (MXBean(java.lang:type=Threading).DaemonThreadCount)',
 'host',
 'Total thread count : (MXBean(java.lang:type=Threading).ThreadCount)',
 'Rel. nonHeap usage : ((MXBean(java.lang:type=Memory).NonHeapMemoryUsage.used / MXBean(java.lang:type=Memory).NonHeapMemoryUsage.max))',
 'Rel. physical mem usage : ((MXBean(java.lang:type=OperatingSystem).FreePhysicalMemorySize / MXBean(java.lang:type=OperatingSystem).TotalPhysicalMemorySize))',
 'GC time : (incld/dx (MXBean(java.lang:name=PS MarkSweep,type=GarbageCollector).CollectionTime))',
 'Rel. Swap Usage : ((\\Swap\\used / \\Swap\\total))' 
 ])

In [4]:
# data_test csvファイルの読み込み
test_df = pd.read_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/test.zip (Unzipped Files)/test.csv',usecols=[
 'GC activity : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionCount))',
 'GC time : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionTime))',
 'Rel. swap usage : ((MXBean(java.lang:type=OperatingSystem).FreeSwapSpaceSize / MXBean(java.lang:type=OperatingSystem).TotalSwapSpaceSize))',
 'Stuck threads : (MXBean(com.bea:Name=ThreadPoolRuntime,Type=ThreadPoolRuntime).StuckThreadCount)',
 'Stuck threads : (MXBean(com.bea:ApplicationRuntime=sys01,Name=default,Type=WorkManagerRuntime).StuckThreadCount)',
 'Process CPU : (MXBean(java.lang:type=OperatingSystem).ProcessCpuLoad)',
 'Rel. heap committed : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.committed / MXBean(java.lang:type=Memory).HeapMemoryUsage.max))',
 'System CPU : (MXBean(java.lang:type=OperatingSystem).SystemCpuLoad)',
 'Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))',
 'Process CPU : (\\Process(java)\\CPU)',
 'Rel. heap usage : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.used / MXBean(java.lang:type=Memory).HeapMemoryUsage.max))',
 'Rel. open file descriptors : ((MXBean(java.lang:type=OperatingSystem).OpenFileDescriptorCount / MXBean(java.lang:type=OperatingSystem).MaxFileDescriptorCount))',
 'Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))',
 'Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))',
 'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
 'Memory space usage : ((MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.committed / MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.max))',
 'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
 'Memory space usage : ((MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.max))',
 'Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))',
 'Daemon thread count : (MXBean(java.lang:type=Threading).DaemonThreadCount)',
 'host',
 'Total thread count : (MXBean(java.lang:type=Threading).ThreadCount)',
 'Rel. nonHeap usage : ((MXBean(java.lang:type=Memory).NonHeapMemoryUsage.used / MXBean(java.lang:type=Memory).NonHeapMemoryUsage.max))',
 'Rel. physical mem usage : ((MXBean(java.lang:type=OperatingSystem).FreePhysicalMemorySize / MXBean(java.lang:type=OperatingSystem).TotalPhysicalMemorySize))',
 'GC time : (incld/dx (MXBean(java.lang:name=PS MarkSweep,type=GarbageCollector).CollectionTime))',
 'Rel. Swap Usage : ((\\Swap\\used / \\Swap\\total))' 
 ])

In [5]:
#スコア0.9296の予測データを読み込み
test_y = pd.read_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/提出用ファイル/xgboost_trained_model_26_param_tuning_test_reloaded6.csv')

In [6]:
#Anomalyのみを取り出し値を四捨五入
test_y=test_y["Anomaly"]
test_y=test_y.round(0).astype(int)

In [7]:
#異常フラグのみのリストを作成
train_y=train_df["Anomaly"]

In [8]:
#異常フラグ、カテゴリを除いたリストを作成
train_x=train_df.drop(["Anomaly","host"],axis=1)
test_x=test_df.drop(["host"],axis=1)

In [9]:
train_x = pd.DataFrame(train_x)
test_x = pd.DataFrame(test_x)

In [10]:
train_y = pd.Series(train_y)
test_y = pd.Series(test_y)

In [11]:
# ラベルエンコーディング（OrdinalEncoder）
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder


oe = preprocessing.OrdinalEncoder()
train_encoded = oe.fit_transform(train_df[['host']].values)
train_decoded = oe.inverse_transform(train_encoded)
test_encoded = oe.fit_transform(test_df[['host']].values)
test_decoded = oe.inverse_transform(test_encoded)

print('エンコード結果: ', [train_encoded,test_encoded])
print('元に戻す: ', [train_decoded,test_decoded])

エンコード結果:  [array([[0.],
       [0.],
       [0.],
       ...,
       [9.],
       [9.],
       [9.]]), array([[0.],
       [0.],
       [0.],
       ...,
       [9.],
       [9.],
       [9.]])]
元に戻す:  [array([['lphost06'],
       ['lphost06'],
       ['lphost06'],
       ...,
       ['lphost18'],
       ['lphost18'],
       ['lphost18']], dtype=object), array([['lphost06'],
       ['lphost06'],
       ['lphost06'],
       ...,
       ['lphost18'],
       ['lphost18'],
       ['lphost18']], dtype=object)]


In [12]:
train_encoded_df = pd.DataFrame(train_encoded)
train_encoded_df.columns=['host']
test_encoded_df = pd.DataFrame(test_encoded)
test_encoded_df.columns=['host']

In [13]:
#エンコードしたカテゴリデータとカテゴリを除いた訓練データを結合
train_x = pd.concat([train_x, train_encoded_df],axis=1)
test_x = pd.concat([test_x, test_encoded_df],axis=1)

In [14]:
#trainデータとtestデータを結合
train2_x = pd.concat([train_x, test_x],axis=0)
train2_y = pd.concat([train_y, test_y],axis=0)

In [15]:
# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train2_x))[0]
x_train, x_valid = train2_x.iloc[tr_idx], train2_x.iloc[va_idx]
y_train, y_valid = train2_y.iloc[tr_idx], train2_y.iloc[va_idx]


In [16]:
print(y_train)

0          0
2          0
3          0
4          0
5          0
          ..
2927226    0
2927228    0
2927229    0
2927230    0
2927231    0
Name: Anomaly, Length: 3754218, dtype: int64


In [17]:
print(x_train.shape)
print(x_valid.shape)
print(test_x.shape)

(3754218, 26)
(938555, 26)
(2927232, 26)


In [18]:
# 学習に使用するデータを設定
#lgb_train = lgb.Dataset(x_train, y_train)
#lgb_eval = lgb.Dataset(x_valid, y_valid,reference=lgb_train)

In [19]:
# xgboostの実装
# -----------------------------------
import xgboost as xgb
from sklearn.metrics import log_loss

# 特徴量と目的変数をxgboostのデータ構造に変換する
dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_valid,label=y_valid)
dtest = xgb.DMatrix(test_x)

In [19]:
'''
#ハイパーパラメータのチューニング
# xgboostによる学習・予測を行うクラス
import xgboost as xgb



class Model:

    def __init__(self, params=None):
        self.model = None
        if params is None:
            self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y, va_x, va_y):
        params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'eta': 0.1,
    'gamma': 0.0,
    'alpha': 0.0,
    'lambda': 1.0,
    'min_child_weight': 1,
    'max_depth': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 71,}
        params.update(self.params)
        num_round = 100000
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        self.model = xgb.train(params, dtrain, num_round, evals=watchlist,early_stopping_rounds=20)

    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred


# -----------------------------------
# 探索するパラメータの空間の指定
# -----------------------------------
# hp.choiceでは、複数の選択肢から選ぶ
# hp.uniformでは、下限・上限を指定した一様分布から抽出する。引数は下限・上限
# hp.quniformでは、下限・上限を指定した一様分布のうち一定の間隔ごとの点から抽出する。引数は下限・上限・間隔
# hp.loguniformでは、下限・上限を指定した対数が一様分布に従う分布から抽出する。引数は下限・上限の対数をとった値

from hyperopt import hp

space = {
    'activation': hp.choice('activation', ['prelu', 'relu']),
    'dropout': hp.uniform('dropout', 0, 0.2),
    'units': hp.quniform('units', 32, 256, 32),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.00001), np.log(0.01)),
}
# hyperoptを使ったパラメータ探索
# -----------------------------------
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import log_loss


def score(params):
    # パラメータを与えたときに最小化する評価指標を指定する
    # 具体的には、モデルにパラメータを指定して学習・予測させた場合のスコアを返すようにする

    # max_depthの型を整数型に修正する
    params['max_depth'] = int(params['max_depth'])

    # Modelクラスを定義しているものとする
    # Modelクラスは、fitで学習し、predictで予測値の確率を出力する
    model = Model(params)
    model.fit(x_train, y_train, x_valid, y_valid)
    va_pred = model.predict(x_valid)
    score = log_loss(y_valid, va_pred)
    print(f'params: {params}, logloss: {score:.4f}')

    # 情報を記録しておく
    history.append((params, score))

    return {'loss': score, 'status': STATUS_OK}


# 探索するパラメータの空間を指定する
space = {
    'min_child_weight': hp.loguniform('min_child_weight', np.log(0.1), np.log(10)),
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'subsample': hp.quniform('subsample', 0.6, 0.95, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.6, 0.95, 0.05),
    'gamma': hp.loguniform('gamma', np.log(1e-8), np.log(1.0)),
    # 余裕があればalpha, lambdaも調整する
    # 'alpha' : hp.loguniform('alpha', np.log(1e-8), np.log(1.0)),
    # 'lambda' : hp.loguniform('lambda', np.log(1e-6), np.log(10.0)),
}
# hyperoptによるパラメータ探索の実行
max_evals = 25
trials = Trials()
history = []
fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

ストリーミング出力は最後の 5000 行に切り捨てられました。

[50]	train-error:0.003355	eval-error:0.003293

[51]	train-error:0.003392	eval-error:0.003323

[52]	train-error:0.002631	eval-error:0.002568

[53]	train-error:0.002732	eval-error:0.002643

[54]	train-error:0.002675	eval-error:0.002602

[55]	train-error:0.00268	eval-error:0.002599

[56]	train-error:0.002686	eval-error:0.0026

[57]	train-error:0.002619	eval-error:0.002537

[58]	train-error:0.002649	eval-error:0.002564

[59]	train-error:0.002601	eval-error:0.002523

[60]	train-error:0.002599	eval-error:0.002524

[61]	train-error:0.002606	eval-error:0.002517

[62]	train-error:0.002602	eval-error:0.002515

[63]	train-error:0.002444	eval-error:0.002367

[64]	train-error:0.002434	eval-error:0.002361

[65]	train-error:0.00243	eval-error:0.002359

[66]	train-error:0.002415	eval-error:0.002332

[67]	train-error:0.002414	eval-error:0.002328

[68]	train-error:0.002433	eval-error:0.002347

[69]	train-error:0.002495	eval-error:0.002419

[70]	train-error:0.002379	eval-

AssertionError: ignored

In [1]:
'''
# 記録した情報からパラメータとスコアを出力する
# （trialsからも情報が取得できるが、パラメータの取得がやや行いづらいため）
history = sorted(history, key=lambda tpl: tpl[1])
best = history[0]
print(f'best params:{best[0]}, score:{best[1]:.8f}')

NameError: ignored

best params:

{'colsample_bytree': 0.6000000000000001, 'gamma': 0.0007438941295364092, 'max_depth': 7, 'min_child_weight': 0.1723663432116852, 'subsample': 0.75}



In [20]:
# 学習データとバリデーションデータのスコアのモニタリング
# -----------------------------------
# モニタリングをloglossで行い、アーリーストッピングの観察するroundを20とする
params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'eta': 0.1,
    'gamma':  0.0007438941295364092,
    'alpha': 0.0,
    'lambda': 1.0,
    'min_child_weight': 0.1723663432116852,
    'max_depth': 7,
    'subsample': 0.75,
    'colsample_bytree': 0.6,
    'random_state': 71,
    #上記で取得したパラメータ
}
num_round = 10000
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
model = xgb.train(params, dtrain, num_round, evals=watchlist,
                  early_stopping_rounds=50)

[0]	train-error:0.002621	eval-error:0.002698
Multiple eval metrics have been passed: 'eval-error' will be used for early stopping.

Will train until eval-error hasn't improved in 50 rounds.
[1]	train-error:0.002517	eval-error:0.002502
[2]	train-error:0.002401	eval-error:0.002446
[3]	train-error:0.003132	eval-error:0.003153
[4]	train-error:0.002794	eval-error:0.002839
[5]	train-error:0.00317	eval-error:0.003173
[6]	train-error:0.003252	eval-error:0.003278
[7]	train-error:0.0032	eval-error:0.003238
[8]	train-error:0.002861	eval-error:0.002896
[9]	train-error:0.002892	eval-error:0.002915
[10]	train-error:0.002587	eval-error:0.00263
[11]	train-error:0.001936	eval-error:0.001935
[12]	train-error:0.001727	eval-error:0.001735
[13]	train-error:0.001471	eval-error:0.001487
[14]	train-error:0.001267	eval-error:0.001277
[15]	train-error:0.001281	eval-error:0.001309
[16]	train-error:0.00113	eval-error:0.001154
[17]	train-error:0.001251	eval-error:0.001267
[18]	train-error:0.001129	eval-error:0.001

In [21]:
# バリデーションデータでのスコアの確認
va_pred = model.predict(dvalid)
score = log_loss(y_valid, va_pred)
print(f'logloss: {score:.12f}')

logloss: nan


In [22]:
# 構築したモデルの保存
import pickle
file = '/content/drive/MyDrive/Nishika ソフトウェアの異常検知/モデル/xgboost_trained_model_26_param_tuning_test_reloaded7.pkl'
pickle.dump(model, open(file, 'wb'))

In [23]:
#from モジュール名 import クラス名（もしくは関数名や変数名）
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#matplotlibをinlineで表示するためのおまじない (plt.show()しなくていい)
%matplotlib inline

In [24]:
# 構築したモデルの読み込み
import pickle
filename = '/content/drive/MyDrive/Nishika ソフトウェアの異常検知/モデル/xgboost_trained_model_26_param_tuning_test_reloaded7.pkl'
loaded_model = pickle.load(open(filename, 'rb'))

In [25]:
# submission csvファイルの読み込み
submission_df = pd.read_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/data.zip (Unzipped Files)/sample_submission.csv')

In [26]:
id_df=submission_df.iloc[:,0]
id_df.head()

0    0
1    1
2    2
3    3
4    4
Name: id, dtype: int64

In [27]:
pred=loaded_model.predict(dtest,ntree_limit=loaded_model.best_ntree_limit)

In [28]:
pred_df = pd.DataFrame(pred)
pred_df.head()

,0
0,2.007277e-08
1,2.226906e-08
2,1.793227e-08
3,1.749623e-08
4,2.226906e-08


In [29]:
output_df=pd.concat([id_df, pred_df],axis=1)
output_df.columns=['id','Anomaly']
output_df.head()

,id,Anomaly
0,0,2.007277e-08
1,1,2.226906e-08
2,2,1.793227e-08
3,3,1.749623e-08
4,4,2.226906e-08


In [30]:
output_df.to_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/提出用ファイル/xgboost_trained_model_26_param_tuning_test_reloaded7.csv',index=False)